<a href="https://colab.research.google.com/github/NurusSafaah/Insight-Learning/blob/main/Insight_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
users = pd.read_excel('https://docs.google.com/spreadsheets/d/1KZQ-dMRtaaz5LYc4BP8gmSfq3EjCDL2H/export?format=xlsx')
exam_result = pd.read_excel('https://docs.google.com/spreadsheets/d/1iEH__ViHfivigP850xBaDfP_uwyEYilN/export?format=xlsx')
exam_regis = pd.read_excel('https://docs.google.com/spreadsheets/d/1fdX6vbFbrfqj4W44GlqouO1YQOgpm-Sk/export?format=xlsx')
dev_jour = pd.read_excel('https://docs.google.com/spreadsheets/d/1Z9Avc3BqV16Q_uFAuL3NvAnww_4aXGD2/export?format=xlsx')
dev_jour_track = pd.read_excel('https://docs.google.com/spreadsheets/d/1LgHYDWxR-op4xPORxrV522366NaLvMKm/export?format=xlsx')
dev_jour_sub = pd.read_excel('https://docs.google.com/spreadsheets/d/1hkyXmoDWVmZpAG0HwWQnJ4vEETzVdrA7/export?format=xlsx')
dev_jour_complet = pd.read_excel('https://docs.google.com/spreadsheets/d/1qFZd9vyoxIWPyCsqTcfKcNNx3bgGfV0H/export?format=xlsx')
dev_jour_tutor = pd.read_excel('https://docs.google.com/spreadsheets/d/1FX1HeZJQWwB66_JMZH2ZdKOIdRd8UvWu/export?format=xlsx')


files = [
    ('users', users),
    ('exam_result', exam_result),
    ('exam_regis', exam_regis),
    ('dev_jour', dev_jour),
    ('dev_jour_track', dev_jour_track),
    ('dev_jour_sub', dev_jour_sub),
    ('dev_jour_complet', dev_jour_complet),
    ('dev_jour_tutor', dev_jour_tutor)
]
files_copy = files.copy()

for name, df in files_copy:
    print(f"\n=== {name} ===")
    print(df.head(3))
    print(df.columns)


=== users ===
          id  display_name                    name                   email  \
0    96989.0      igihcksn        Inggih Wicaksono      igihcksn@gmail.com   
1   938276.0  nurrizkiadip  Nur Rizki Adi Prasetyo     nrizki@dicoding.com   
2  5021477.0   rifath_2SXp                  rifath  rifathali088@gmail.com   

          phone  user_role  user_verification_status          created_at  \
0  6.281271e+12        2.0                       1.0 2017-04-03 19:16:35   
1  6.287795e+12        2.0                       1.0 2020-05-28 20:13:50   
2  6.289662e+12        2.0                       1.0 2025-05-26 14:14:10   

           updated_at  deleted_at  ... city_id custom_city  \
0 2025-04-28 11:54:11         NaN  ...   428.0         NaN   
1 2025-10-16 10:06:43         NaN  ...    33.0         NaN   
2 2025-10-08 13:17:19         NaN  ...    22.0         NaN   

                           unsubscribe_link            tz         verified_at  \
0  b6b33e749120c67873d2470c6f0abe4627

In [ ]:
# Ganti nama kolom ID di Users agar sesuai dengan FK
users.rename(columns={'id': 'user_id'}, inplace=True)

# Ganti nama kolom submitter_id di submissions
dev_jour_sub.rename(columns={'submitter_id': 'user_id'}, inplace=True)

# Ganti nama kolom developer_id di trackings
dev_jour_track.rename(columns={'developer_id': 'user_id'}, inplace=True)

# Ganti nama kolom examinees_id di exam_regis
exam_regis.rename(columns={'examinees_id': 'user_id'}, inplace=True)

# Ganti nama kolom developer_journey_id di tutorials
dev_jour_tutor.rename(columns={'developer_journey_id': 'journey_id'}, inplace=True)

print("✅ Penamaan ulang kolom kunci selesai.")
for name, df in files_copy:
    print(f"\n=== {name} ===")
    print(df.info)
    print(df.columns)

✅ Penamaan ulang kolom kunci selesai.

=== users ===
<bound method DataFrame.info of       user_id               display_name                    name  \
0     96989.0                   igihcksn        Inggih Wicaksono   
1    938276.0               nurrizkiadip  Nur Rizki Adi Prasetyo   
2   5021477.0                rifath_2SXp                  rifath   
3   5044844.0           ledis_idola_h8Ge             LEDIS IDOLA   
4   5051374.0                   fkaslana        Fircan Ferdinand   
5   5181638.0         anggit_andreansyah      Anggit andreansyah   
6   5410562.0           jeni_amanda_ABFC             Jeni Amanda   
7   5410865.0  ramadhan_oktarizaldi_02Qj    ramadhan oktarizaldi   
8         NaN                        NaN                     NaN   
9         NaN                        NaN                     NaN   
10        NaN                        NaN                     NaN   
11        NaN                        NaN                     NaN   
12        NaN                  

In [ ]:
# Daftar semua DataFrame yang memiliki kolom tanggal/waktu
dfs_to_process = [
    users, exam_result, exam_regis, dev_jour_track, dev_jour_sub,
    dev_jour_complet, dev_jour_tutor
]

# Daftar kolom yang kita yakini berisi data waktu
date_cols_generic = [
    'created_at', 'updated_at', 'deleted_at', 'verified_at',
    'enrollments_at', 'last_enrolled_at', 'started_review_at',
    'ended_review_at', 'first_opened_at', 'last_viewed',
    'completed_at', 'deadline_at', 'retake_limit_at',
    'exam_finished_at', 'look_report_at'
]

print("\n--- Mulai Konversi Tipe Data Waktu ---")

for i, df in enumerate(dfs_to_process):
    df_name = files[i][0]

    for col in date_cols_generic:
        if col in df.columns:
            # errors='coerce' akan mengubah nilai yang tidak valid menjadi NaT (Not a Time)
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Konversi spesifik submission_duration, study_duration, dll. jika ada
    # Asumsi: Kolom skor/durasi numerik sudah terformat (decimal(12,2))
    if 'score' in df.columns:
        df['score_numeric'] = pd.to_numeric(df['score'], errors='coerce')
        df['score_numeric'].fillna(0, inplace=True) # Isi NaN skor dengan 0

    print(f"Konversi datetime selesai untuk {df_name}.")

print("✅ Konversi Tipe Data Waktu selesai.")


--- Mulai Konversi Tipe Data Waktu ---
Konversi datetime selesai untuk users.
Konversi datetime selesai untuk exam_result.
Konversi datetime selesai untuk exam_regis.
Konversi datetime selesai untuk dev_jour.
Konversi datetime selesai untuk dev_jour_track.
Konversi datetime selesai untuk dev_jour_sub.
Konversi datetime selesai untuk dev_jour_complet.
✅ Konversi Tipe Data Waktu selesai.


/tmp/ipython-input-3852893279.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['score_numeric'].fillna(0, inplace=True) # Isi NaN skor dengan 0
/tmp/ipython-input-3852893279.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/tmp/ipython-input-3852893279.py:24: FutureWarning: In a future version of pandas, parsing datetimes with m

In [ ]:
try:
    # Coba filter berdasarkan nilai numerik 2
    users_filtered = users[users['user_role'] == 2].copy()
    print(f"\nJumlah pengguna sebelum filter: {len(users)}")
    print(f"Jumlah Siswa (user_role = 2) setelah filter: {len(users_filtered)}")

except KeyError:
    # Jika kolom user_role berisi string (misalnya 'Siswa')
    users_filtered = users[users['user_role'] == 'Siswa'].copy()
    print(f"\nJumlah pengguna sebelum filter: {len(users)}")
    print(f"Jumlah Siswa (user_role = 'Siswa') setelah filter: {len(users_filtered)}")

# Update DataFrame users ke versi yang sudah difilter
users = users_filtered
print("✅ Pemfilteran peran selesai.")


Jumlah pengguna sebelum filter: 22
Jumlah Siswa (user_role = 2) setelah filter: 8
✅ Pemfilteran peran selesai.


In [ ]:
# 1. Cek Dataframe Users (Pastikan 'user_id' dan 'created_at' sudah benar)
print("--- 👤 Verifikasi Data Users ---")
print(users.info())
print("\nContoh Kolom User ID:")
print(users[['user_id', 'user_role']].head(3))
print("------------------------------")


# 2. Cek Dataframe Tracking (Pastikan 'user_id', 'journey_id', dan kolom waktu sudah benar)
print("--- 🧭 Verifikasi Data Trackings ---")
print(dev_jour_track.info())
print("\nContoh Kolom Waktu di Trackings:")
# Tampilkan kolom 'first_opened_at' atau 'last_viewed'
print(dev_jour_track[['user_id', 'first_opened_at', 'last_viewed']].head(3))
print("-----------------------------------")

--- 👤 Verifikasi Data Users ---
<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 7
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   user_id                     8 non-null      float64       
 1   display_name                8 non-null      object        
 2   name                        8 non-null      object        
 3   email                       8 non-null      object        
 4   phone                       7 non-null      float64       
 5   user_role                   8 non-null      float64       
 6   user_verification_status    8 non-null      float64       
 7   created_at                  8 non-null      datetime64[ns]
 8   updated_at                  8 non-null      datetime64[ns]
 9   deleted_at                  0 non-null      datetime64[ns]
 10  remember_token              5 non-null      object        
 11  image_path                  8 non-n

In [ ]:
# Uji Coba: Gabungkan Users dengan Trackings
# Pilih kolom user_id dan user_role dari users
df_test = users[['user_id', 'user_role']].copy()

# Gabungkan dengan jumlah total trackings per user
df_track_count = dev_jour_track.groupby('user_id')['id'].count().reset_index()
df_track_count.rename(columns={'id': 'total_tracks'}, inplace=True)

# Lakukan Left Merge
df_merged_test = df_test.merge(df_track_count, on='user_id', how='left')

print("\n--- 🧪 Uji Coba Penggabungan Dasar (Users + Trackings) ---")
print(f"Jumlah baris Users awal: {len(users)}")
print(f"Jumlah baris setelah merge: {len(df_merged_test)}")
print(df_merged_test.head())
print("---------------------------------------------------------")


--- 🧪 Uji Coba Penggabungan Dasar (Users + Trackings) ---
Jumlah baris Users awal: 8
Jumlah baris setelah merge: 8
     user_id  user_role  total_tracks
0    96989.0        2.0          3069
1   938276.0        2.0          2205
2  5021477.0        2.0           187
3  5044844.0        2.0           960
4  5051374.0        2.0           157
---------------------------------------------------------


In [ ]:
# Pastikan kita menggunakan user_id dari Users yang sudah difilter
df_master = users[['user_id', 'created_at']].copy()
# Kita akan merge semua agregasi ke df_master ini

# Gabungkan exam_result dan exam_regis menggunakan exam_registration_id
df_exam = exam_regis.merge(exam_result, left_on='id', right_on='exam_registration_id', how='inner')
df_exam.rename(columns={'examinees_id': 'user_id', 'created_at_x': 'reg_date'}, inplace=True)

# Agregasi ke level Siswa: Ambil skor tertinggi dan total ujian yang diambil
df_exam_agg = df_exam.groupby('user_id').agg(
    highest_score=('score_numeric', 'max'),
    total_exams=('id_x', 'count'),
    first_exam_reg_date=('reg_date', 'min')
).reset_index()

df_master = df_master.merge(df_exam_agg, on='user_id', how='left')

In [ ]:
df_tracking_agg = dev_jour_track.groupby('user_id').agg(
    total_tracks=('id', 'count'),
    days_active=('last_viewed', lambda x: x.dt.date.nunique()),
    first_activity=('first_opened_at', 'min'),
    last_activity=('last_viewed', 'max')
).reset_index()

df_master = df_master.merge(df_tracking_agg, on='user_id', how='left')

# Fitur Konsistensi Dasar
df_master['active_duration_days'] = (df_master['last_activity'] - df_master['first_activity']).dt.days.fillna(0)
# Hindari pembagian dengan nol
df_master['consistency_score'] = df_master['days_active'] / df_master['active_duration_days'].replace(0, 1)

In [ ]:
# Hitung Submission Sukses ('diterima' = 1, 'ditolak' = 0)
dev_jour_sub['is_successful'] = np.where(dev_jour_sub['status'] == 'diterima', 1, 0)

df_sub_agg = dev_jour_sub.groupby('user_id').agg(
    num_submissions=('id', 'count'),
    successful_submissions=('is_successful', 'sum'),
    avg_submission_rating=('rating', 'mean'),
    first_submission=('created_at', 'min')
).reset_index()

df_master = df_master.merge(df_sub_agg, on='user_id', how='left', suffixes=('_exam', '_sub'))

# Fitur Refleksi dan Kecepatan
df_master['submission_success_ratio'] = (df_master['successful_submissions_sub'] / df_master['num_submissions_sub']).fillna(0)
df_master['revisiting_ratio'] = df_master['num_submissions_sub'] / df_master['days_active'].replace(0, 1)

# Kecepatan: Waktu yang dibutuhkan untuk submission pertama (dari pendaftaran)
df_master['time_to_first_submit_days'] = (df_master['first_submission_sub'] - df_master['created_at']).dt.days.fillna(0)

print("\n--- Updated df_master with Submission Features ---")
display(df_master.head())
print("----------------------------------------------------")

KeyError: 'successful_submissions_sub'

In [ ]:
# Kolom yang hanya berisi NaN jika siswa tidak aktif/submit
cols_to_fill_zero = ['highest_score', 'total_exams', 'total_tracks', 'days_active',
                     'num_submissions_sub', 'successful_submissions_sub', 'avg_submission_rating_sub']

df_master[cols_to_fill_zero] = df_master[cols_to_fill_zero].fillna(0)

# Hapus kolom waktu mentah
cols_to_drop = ['created_at', 'first_activity', 'last_activity', 'first_submission_sub', 'first_exam_reg_date']
df_master.drop(columns=[col for col in cols_to_drop if col in df_master.columns], inplace=True)

print("✅ Feature Engineering Selesai. df_master siap untuk Pelabelan.")
print(df_master.head())

In [ ]:
# Hitung ambang batas berdasarkan data yang ada
Q25_speed = df_master['time_to_first_submit_days'].quantile(0.25)
Q75_consistency = df_master['consistency_score'].quantile(0.75)
Q75_revisit = df_master['revisiting_ratio'].quantile(0.75)

# Ambang batas Skor (dapat disesuaikan)
SCORE_HIGH_THRESHOLD = 80
SCORE_MID_THRESHOLD = 60

In [ ]:
def label_learner_type(row):
    # Kriteria 1: Fast Learner (Sangat Cepat DAN Skor Tinggi)
    # Waktu submit kecil (cepat) AND Skor tinggi
    if (row['time_to_first_submit_days'] <= Q25_speed) and (row['highest_score'] >= SCORE_HIGH_THRESHOLD):
        return 'Fast Learner'

    # Kriteria 2: Consistent Learner (Sangat Konsisten DAN Skor Menengah/Baik)
    # Konsistensi tinggi AND Skor di atas batas tengah
    elif (row['consistency_score'] >= Q75_consistency) and (row['highest_score'] >= SCORE_MID_THRESHOLD):
        return 'Consistent Learner'

    # Kriteria 3: Reflective Learner (Revisi Tinggi/Sering Coba DAN Skor Menengah/Baik)
    # Rasio revisit tinggi AND Rasio sukses submission cukup baik
    elif (row['revisiting_ratio'] >= Q75_revisit) and (row['submission_success_ratio'] > 0.5) and (row['highest_score'] >= SCORE_MID_THRESHOLD):
        return 'Reflective Learner'

    else:
        return 'Other Learner'

# Terapkan fungsi untuk membuat kolom target 'learner_type'
df_master['learner_type'] = df_master.apply(label_learner_type, axis=1)

print("\n--- Distribusi Kelas Target ---")
print(df_master['learner_type'].value_counts())

In [ ]:
users.merge(dev_jour_complet, left_on='id', right_on='user_id', how='left').merge(dev_jour_track, left_on='id', right_on='developer_id', how='left').merge(dev_jour_sub, left_on='id', right_on='submitter_id', how='left').merge(exam_regis, left_on='id', right_on='examinees_id', how='left').merge(exam_result, left_on='id_y', right_on='exam_registration_id', how='left')
